In [85]:
# ! pip install pyspark

In [11]:
# IMporting the libraries
import pyspark


# Use 3 features:  'Type', 'Age', 'Breed1'

## Using pyspark to read the data and process it

In [43]:
# To work with spark we need to create a spark session
# Need to instal java
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('petfinder').getOrCreate()

########### For the train dataset
# Read a dataset with spark
df_spark = spark.read.csv('./train_balanced_corr.csv', header=True, inferSchema=True)
# Header = True, inferSchema = True, means that the first row is the header and the schema is inferred (if schema is not inferred, all columns will be read as string)
# Convert the column "AdoptionSpeed" to integer
df_spark = df_spark.withColumn("AdoptionSpeed", df_spark["AdoptionSpeed"].cast("integer"))


############ For the test dataset
# Read a dataset with spark
df_spark_test = spark.read.csv('./test_split_corr.csv', header=True, inferSchema=True)
# Header = True, inferSchema = True, means that the first row is the header and the schema is inferred (if schema is not inferred, all columns will be read as string)
# Convert the column "AdoptionSpeed" to integer
df_spark_test = df_spark_test.withColumn("AdoptionSpeed", df_spark_test["AdoptionSpeed"].cast("integer"))


## Data cleaning

In [44]:

########### For the train dataset

## Drop rows with missing values
# df_spark.na.drop(how='all', thresh=10).show() 
    ### how='any' means drop rows with any missing value, how='all' means drop rows whose all values are missing
    ### thresh=10 means drop rows whose number of missing values is greater than 10
    ### subset=['Age'] means drop rows whose 'Age' value is missing
df_spark = df_spark.na.drop(how= 'any' , subset=['AdoptionSpeed'])
## Fill missing values with mean
from pyspark.sql.functions import mean
mean_val = df_spark.select(mean(df_spark['Age'])).collect()
mean_age = mean_val[0][0]
df_spark.na.fill(mean_age, subset=['Age']).show()

########### For the test dataset

df_spark_test = df_spark_test.na.drop(how= 'any' , subset=['AdoptionSpeed'])
## Fill missing values with mean
from pyspark.sql.functions import mean
mean_val = df_spark_test.select(mean(df_spark_test['Age'])).collect()
mean_age = mean_val[0][0]
df_spark_test.na.fill(mean_age, subset=['Age']).show()




+------+---+----+-------------+
|Breed1|Age|Type|AdoptionSpeed|
+------+---+----+-------------+
|   307|  2|   1|            1|
|   307| 36|   1|            4|
|   179|  2|   1|            1|
|   265| 27|   2|            4|
|   307|  2|   1|            1|
|   266| 29|   2|            4|
|    83| 36|   1|            1|
|   307| 24|   1|            3|
|   307| 21|   1|            4|
|   307| 29|   1|            3|
|   307|  3|   1|            2|
|    60|120|   1|            2|
|   145| 18|   1|            2|
|   254| 24|   2|            3|
|   205| 36|   1|            2|
|   307|  2|   1|            3|
|   266| 12|   2|            3|
|   307|  3|   1|            2|
|   283| 48|   2|            4|
|   265|  3|   2|            2|
+------+---+----+-------------+
only showing top 20 rows

+------+---+----+-------------+
|Breed1|Age|Type|AdoptionSpeed|
+------+---+----+-------------+
|   265|  7|   2|            4|
|   266| 24|   2|            4|
|   266| 12|   2|            2|
|   195| 60|  

# Using PySpark MLlib to build the model

In [45]:
# First, collect the features in a single column

from pyspark.ml.feature import VectorAssembler

#### For the train dataset
featureassemble = VectorAssembler(inputCols=['Breed1','Age','Type'], outputCol='features')
output = featureassemble.transform(df_spark) # This will create a new column called 'features' which is a vector of the selected columns (Type, Age2, Breed1) by the VectorAssembler
output.show()

#### For the test dataset
testfeatureassemble = VectorAssembler(inputCols=['Breed1','Age','Type'], outputCol='features')
testoutput = testfeatureassemble.transform(df_spark_test) # This will create a new column called 'features' which is a vector of the selected columns (Type, Age2, Breed1) by the VectorAssembler
testoutput.show()


+------+---+----+-------------+----------------+
|Breed1|Age|Type|AdoptionSpeed|        features|
+------+---+----+-------------+----------------+
|   307|  2|   1|            1| [307.0,2.0,1.0]|
|   307| 36|   1|            4|[307.0,36.0,1.0]|
|   179|  2|   1|            1| [179.0,2.0,1.0]|
|   265| 27|   2|            4|[265.0,27.0,2.0]|
|   307|  2|   1|            1| [307.0,2.0,1.0]|
|   266| 29|   2|            4|[266.0,29.0,2.0]|
|    83| 36|   1|            1| [83.0,36.0,1.0]|
|   307| 24|   1|            3|[307.0,24.0,1.0]|
|   307| 21|   1|            4|[307.0,21.0,1.0]|
|   307| 29|   1|            3|[307.0,29.0,1.0]|
|   307|  3|   1|            2| [307.0,3.0,1.0]|
|    60|120|   1|            2|[60.0,120.0,1.0]|
|   145| 18|   1|            2|[145.0,18.0,1.0]|
|   254| 24|   2|            3|[254.0,24.0,2.0]|
|   205| 36|   1|            2|[205.0,36.0,1.0]|
|   307|  2|   1|            3| [307.0,2.0,1.0]|
|   266| 12|   2|            3|[266.0,12.0,2.0]|
|   307|  3|   1|   

In [46]:
# Select the features and the target column

#### For the train dataset
finalized_data = output.select('features', 'AdoptionSpeed') # Select the features and the target column
finalized_data.show()

#### For the test dataset
testfinalized_data = testoutput.select('features', 'AdoptionSpeed') # Select the features and the target column
testfinalized_data.show()


+----------------+-------------+
|        features|AdoptionSpeed|
+----------------+-------------+
| [307.0,2.0,1.0]|            1|
|[307.0,36.0,1.0]|            4|
| [179.0,2.0,1.0]|            1|
|[265.0,27.0,2.0]|            4|
| [307.0,2.0,1.0]|            1|
|[266.0,29.0,2.0]|            4|
| [83.0,36.0,1.0]|            1|
|[307.0,24.0,1.0]|            3|
|[307.0,21.0,1.0]|            4|
|[307.0,29.0,1.0]|            3|
| [307.0,3.0,1.0]|            2|
|[60.0,120.0,1.0]|            2|
|[145.0,18.0,1.0]|            2|
|[254.0,24.0,2.0]|            3|
|[205.0,36.0,1.0]|            2|
| [307.0,2.0,1.0]|            3|
|[266.0,12.0,2.0]|            3|
| [307.0,3.0,1.0]|            2|
|[283.0,48.0,2.0]|            4|
| [265.0,3.0,2.0]|            2|
+----------------+-------------+
only showing top 20 rows

+----------------+-------------+
|        features|AdoptionSpeed|
+----------------+-------------+
| [265.0,7.0,2.0]|            4|
|[266.0,24.0,2.0]|            4|
|[266.0,12.0,2.0]

### 1. Logistic Regression

In [49]:
from pyspark.ml.classification import LogisticRegression
# Split the data into training and validation data
train_data = finalized_data
classifier = LogisticRegression
classifier = LogisticRegression(labelCol='AdoptionSpeed').fit(train_data) # Fit the model


test_data = testfinalized_data
results = classifier.evaluate(test_data) # Evaluate the model on the validation data
results.predictions.show() # Show the predictions
results.predictions.select('AdoptionSpeed', 'prediction').show() # Show the target and the prediction

# Want to show the f1 score and confusion matrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="AdoptionSpeed", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(results.predictions)
print("F1 score: %.3f" % f1_score)

# Confusion matrix
results.predictions.crosstab('AdoptionSpeed', 'prediction').show()



+----------------+-------------+--------------------+--------------------+----------+
|        features|AdoptionSpeed|       rawPrediction|         probability|prediction|
+----------------+-------------+--------------------+--------------------+----------+
| [265.0,7.0,2.0]|            4|[0.21852222769436...|[0.24594501729175...|       0.0|
|[266.0,24.0,2.0]|            4|[0.21091546173330...|[0.24452052550620...|       0.0|
|[266.0,12.0,2.0]|            2|[0.21328101316232...|[0.24531022067589...|       0.0|
|[195.0,60.0,1.0]|            1|[0.02040640887454...|[0.19880089651182...|       4.0|
| [266.0,3.0,2.0]|            4|[0.21505517673408...|[0.24453461523033...|       0.0|
|[307.0,12.0,1.0]|            2|[-0.4467550129775...|[0.12267623478828...|       4.0|
|[218.0,16.0,1.0]|            4|[-0.0687979689279...|[0.18565850870324...|       3.0|
|[285.0,24.0,2.0]|            3|[0.13005966777085...|[0.22530280619895...|       4.0|
|[266.0,36.0,2.0]|            4|[0.20854991030428...|[

### 2. Decision Tree

In [52]:
from pyspark.ml.classification import DecisionTreeClassifier
# Split the data into training and validation data
train_data = finalized_data
classifier = DecisionTreeClassifier( labelCol='AdoptionSpeed', featuresCol='features')
classifier = classifier.fit(train_data) # Fit the model


test_data = testfinalized_data
results = classifier.transform(test_data) # Evaluate the model on the validation data
results.show() # Show the predictions
results.select('AdoptionSpeed', 'prediction').show() # Show the target and the prediction

# Want to show the f1 score and confusion matrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="AdoptionSpeed", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(results)
print("F1 score: %.3f" % f1_score)

# Confusion matrix
results.crosstab('AdoptionSpeed', 'prediction').show()



+----------------+-------------+--------------------+--------------------+----------+
|        features|AdoptionSpeed|       rawPrediction|         probability|prediction|
+----------------+-------------+--------------------+--------------------+----------+
| [265.0,7.0,2.0]|            4|[388.0,258.0,317....|[0.19715447154471...|       4.0|
|[266.0,24.0,2.0]|            4|[388.0,258.0,317....|[0.19715447154471...|       4.0|
|[266.0,12.0,2.0]|            2|[388.0,258.0,317....|[0.19715447154471...|       4.0|
|[195.0,60.0,1.0]|            1|[380.0,195.0,147....|[0.39915966386554...|       0.0|
| [266.0,3.0,2.0]|            4|[475.0,657.0,581....|[0.19467213114754...|       1.0|
|[307.0,12.0,1.0]|            2|[163.0,156.0,291....|[0.08101391650099...|       4.0|
|[218.0,16.0,1.0]|            4|[388.0,258.0,317....|[0.19715447154471...|       4.0|
|[285.0,24.0,2.0]|            3|[214.0,128.0,98.0...|[0.31845238095238...|       0.0|
|[266.0,36.0,2.0]|            4|[388.0,258.0,317....|[

### 3. Random Forest

In [53]:
from pyspark.ml.classification import RandomForestClassifier

train_data = finalized_data
classifier = RandomForestClassifier(labelCol='AdoptionSpeed', featuresCol='features')
classifier = classifier.fit(train_data) # Fit the model


test_data = testfinalized_data
results = classifier.transform(test_data) # Evaluate the model on the validation data
results.show() # Show the predictions
results.select('AdoptionSpeed', 'prediction').show() # Show the target and the prediction

# Want to show the f1 score and confusion matrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="AdoptionSpeed", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(results)
print("F1 score: %.3f" % f1_score)

# Confusion matrix
results.crosstab('AdoptionSpeed', 'prediction').show()



+----------------+-------------+--------------------+--------------------+----------+
|        features|AdoptionSpeed|       rawPrediction|         probability|prediction|
+----------------+-------------+--------------------+--------------------+----------+
| [265.0,7.0,2.0]|            4|[4.30130519439516...|[0.21506525971975...|       4.0|
|[266.0,24.0,2.0]|            4|[3.89876794043720...|[0.19493839702186...|       4.0|
|[266.0,12.0,2.0]|            2|[3.89876794043720...|[0.19493839702186...|       4.0|
|[195.0,60.0,1.0]|            1|[7.44879126264556...|[0.37243956313227...|       0.0|
| [266.0,3.0,2.0]|            4|[4.23385838023537...|[0.21169291901176...|       1.0|
|[307.0,12.0,1.0]|            2|[1.64190304309554...|[0.08209515215477...|       4.0|
|[218.0,16.0,1.0]|            4|[4.56098088565224...|[0.22804904428261...|       4.0|
|[285.0,24.0,2.0]|            3|[4.81435579691729...|[0.24071778984586...|       4.0|
|[266.0,36.0,2.0]|            4|[3.65950492337921...|[

### 4. Naive Bayes

In [54]:
from pyspark.ml.classification import NaiveBayes

train_data = finalized_data
classifier = NaiveBayes(labelCol='AdoptionSpeed', featuresCol='features')
classifier = classifier.fit(train_data) # Fit the model


test_data = testfinalized_data
results = classifier.transform(test_data) # Evaluate the model on the validation data
results.show() # Show the predictions
results.select('AdoptionSpeed', 'prediction').show() # Show the target and the prediction

# Want to show the f1 score and confusion matrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="AdoptionSpeed", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(results)
print("F1 score: %.3f" % f1_score)

# Confusion matrix
results.crosstab('AdoptionSpeed', 'prediction').show()


+----------------+-------------+--------------------+--------------------+----------+
|        features|AdoptionSpeed|       rawPrediction|         probability|prediction|
+----------------+-------------+--------------------+--------------------+----------+
| [265.0,7.0,2.0]|            4|[-46.974540850384...|[0.14402667045828...|       2.0|
|[266.0,24.0,2.0]|            4|[-101.24812790298...|[0.17873348172516...|       4.0|
|[266.0,12.0,2.0]|            2|[-62.971521419971...|[0.25457037136800...|       0.0|
|[195.0,60.0,1.0]|            1|[-207.53567000969...|[0.00110149909586...|       4.0|
| [266.0,3.0,2.0]|            4|[-34.264066557712...|[0.06978615073014...|       2.0|
|[307.0,12.0,1.0]|            2|[-59.849431997550...|[0.19599104897496...|       3.0|
|[218.0,16.0,1.0]|            4|[-68.301187210301...|[0.24240190122342...|       4.0|
|[285.0,24.0,2.0]|            3|[-102.16762406796...|[0.19066671534541...|       4.0|
|[266.0,36.0,2.0]|            4|[-139.52473438599...|[